In [ ]:
# For licensing see accompanying LICENSE file.
# Copyright (C) 2025 Apple Inc. All Rights Reserved.
# get human annotation data as csv

import pandas as pd

df = pd.read_csv("../../data/human/arduin-long-form-factuality-20240806-2255_(uep-119589).csv")

In [ ]:
# get original data that given to annotators
# (this includes the ground-truth preference)

import json

with open("../../data/generated/longfact/individual_generations_v6_addmore/longfact-initial-test-20240805.jsonl") as f:
    data = json.load(f)
    
df_gt = pd.json_normalize(data) # this is a highly nested json, normalize it

In [ ]:
# merge both datasets on prompt id

full_df = pd.merge(df, df_gt, left_on="Prompt Id", right_on="prompt_id")

In [ ]:
# get human preference from OPR rating

def get_human_pref(row):
    rating = row["Satisfaction OPR text_b,text_a"]
    if rating < 0:
        return "text_b"
    elif rating > 0:
        return "text_a"
    else:
        return "tie"

full_df["converted_ratings"] = full_df.apply(get_human_pref, axis=1)

# add columns with intuitive names
full_df[["human_pref", "gt_pref"]] = full_df[["converted_ratings", "enrichments.meta.longfact-preferred"]]

# add 
full_df["agreed"] = full_df["human_pref"] == full_df["gt_pref"]

In [ ]:
# get value counts of agreement
full_df.agreed.value_counts()

In [ ]:
# remove ties, and get value counts
filtered_df = full_df.copy()
filtered_df = filtered_df[filtered_df["human_pref"] != "tie"]
filtered_df.agreed.value_counts()